In [ ]:
# Evaluate Site 3 detections against an input json label file 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
module_path = module_path + '/code/'
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json 
import renderapi
import scipy
import numpy as np
from scipy.stats import norm
from renderapps.module.render_module import RenderModule

import processDetections as pd
import matplotlib
import matplotlib.pyplot as plt
from shapely import geometry
from skimage import measure
import synaptogram
from scipy import io
import SynapseDetection as syn

In [ ]:
args={
    'render':{
    'host':'http://152.3.214.113',
    'port':8080,
    'owner':'Anish',
    'project':'M247514_Rorb_1',
    'client_scripts':'/home/anish/Connectome/render/render-ws-java-client/src/main/scripts'
},
'log_level':'DEBUG'}
mod = RenderModule(input_data = args,args=[])
stacks = mod.render.run(renderapi.render.get_stacks_by_owner_project)
render_args = args['render']

In [ ]:
metadataFN = 'metadatatest.json'
metadata = syn.loadMetadata(metadataFN)
n = 0 
datalocation = metadata['datalocation']
outputFN = os.path.join(datalocation, 'resultVol')
outputFN = outputFN + str(n) + '.json'

In [ ]:
annotationdata = json.load(open('json_annotations/m247514_Site3Annotation_MN_global_v2.json'))


In [ ]:
detectiondata = json.load(open(outputFN))

In [ ]:
annotationList = annotationdata['area_lists']
detectionList = detectiondata['area_lists']

In [ ]:
print len(annotationList)
print len(detectionList)

In [ ]:
detectionsExpanded = []

itr = 0
for detection in detectionList: 
    bboxCoordinates = synaptogram.getAnnotationBoundingBox2(detection, render_args)
    expandedBox = synaptogram.expandBoundingBox(bboxCoordinates, win_xy=4, win_z=1)
    detectionpoly = pd.createPolygonFromBBox(expandedBox)
    zlistdetection = synaptogram.getZListFromBoundingBox(expandedBox)

    zListAndPoly = {'zlist': zlistdetection, 'poly': detectionpoly}
    detectionsExpanded.append(zListAndPoly)

    

In [ ]:
synapsesExpanded = []

for synapse in annotationList: 
    bboxCoordinates = synaptogram.getAnnotationBoundingBox2(synapse, render_args)
    bboxCoordinates = synaptogram.transformSynapseCoordinates(bboxCoordinates)
    expandedBox = synaptogram.expandBoundingBox(bboxCoordinates, 1, 0)
    synapsepoly = pd.createPolygonFromBBox(expandedBox)
    zlistsynapse = synaptogram.getZListFromBoundingBox(expandedBox)

    zListAndPoly = {'zlist': zlistsynapse, 'poly': synapsepoly}
    synapsesExpanded.append(zListAndPoly)

In [ ]:
overlapMatrix = np.zeros((len(detectionsExpanded), len(synapsesExpanded)))

cItr = 0
for synapse in synapsesExpanded: 

    zlistSynapse = synapse['zlist']
    polySynapse = synapse['poly']

    rItr = 0 
    for detection in detectionsExpanded: 

        zlistDetection = detection['zlist']
        polydetection = detection['poly']

        if not set(zlistDetection).isdisjoint(zlistSynapse):
            if polySynapse.intersects(polydetection): 
                overlapMatrix[rItr, cItr] = 1

        rItr = rItr + 1
    cItr = cItr + 1
    

In [ ]:
synapseOverlap = (np.sum(overlapMatrix, 0))

In [ ]:
np.count_nonzero(synapseOverlap) / float(len(synapseOverlap))

In [ ]:
np.count_nonzero(synapseOverlap)

In [ ]:
float(len(synapseOverlap))

In [ ]:
# create synaptograms with detections 

In [ ]:
synapse = annotationList[6]
win_xy = 4
win_z = 1
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/';
stackList = ['results',  'PSD95', 'synapsin', 'VGlut1', 'GluN1', 'GABA', 'Gephyrin']
showProb = True
textXOffset = 0
textYOffset = 5


